## 从github中读取代码库
通过`GithubFileLoader`方法直接从指定的仓库中读取文件。

In [ ]:
from langchain_community.document_loaders import GithubFileLoader
import os
from dotenv import load_dotenv
load_dotenv()

# 替换为你自己的仓库，或者一个公开的小仓库
# 注意：如果是私有仓库，需要配置 GITHUB_ACCESS_TOKEN
loader = GithubFileLoader(
    repo="Xheng222/RISC-V-CPU",  # 示例仓库
    access_token=os.getenv("GITHUB_ACCESS_TOKEN"),
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(".v") # 先只读 Markdown 文件试试
)

documents = loader.load()
print(f"Loaded {len(documents)} documents")
print("1: ",documents[0].page_content[:500])
print("2: ",documents[1].page_content[:500])
print("3: ",documents[2].page_content[:500])

## LLM
初始化LLM，进行简单的文本分割并定义prompt

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 1. 初始化 LLM
llm = ChatOpenAI(model="deepseek-chat", temperature=0)

# 2. 简单的文本分割 (Verilog 没有专门的 splitter，用通用的即可)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
# 选取第一个文件做测试
test_doc = documents[0] 
splits = splitter.split_documents([test_doc])

# 3. 定义审查 Prompt
prompt = ChatPromptTemplate.from_template("""
你是一位资深的数字集成电路设计专家。请审查以下 Verilog 代码：

文件名: {filename}

代码内容:
```verilog
{code}
请提供以下反馈：

1. 功能摘要：这段代码实现了什么功能？
2. 潜在Bug：是否存在时序违例、锁存器推断风险或逻辑错误？
3. 代码规范：命名是否规范，注释是否清晰？
4. 优化建议：如何优化面积或功耗？
""")

### 构建并运行链

In [ ]:
chain = prompt | llm

In [ ]:
print(f"正在审查文件：{test_doc.metadata['source']}，共 {len(splits)} 个代码片段。")
response = chain.invoke({
    "filename":test_doc.metadata['source'],
    "code": splits[0].page_content # 仅审查第一个分片
})

print("审查结果：")
print(response.content)